In [10]:
import pandas as pd
import numpy as np
import glob
import os
import geopandas as gpd
import urllib
import sys
import pathlib
import glob
import matplotlib.pyplot as plt
from urllib.parse import quote
from sqlalchemy import create_engine
import configparser

import geopandas as gpd
import shapely

import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px
from fluxdataqaqc import Data, QaQc, Plot
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import LinearAxis, Range1d
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
sys.path.append("//")
#sys.path.append("../../Micromet")
import micromet
from micromet import AmerifluxDataProcessor
%matplotlib inline

In [3]:
from pyproj import Transformer
EPSG = 5070
# load initial flux data
station = 'US-UTW'
config_path = f'../../station_config/{station}.ini'
config = configparser.ConfigParser()
config.read(config_path)

latitude = config['METADATA']['station_latitude']
longitude = config['METADATA']['station_longitude']

# Define the transformer from WGS84 to NAD83 Conus Albers
transformer = Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)

# Perform the transformation
station_x, station_y = transformer.transform(longitude, latitude)

print(f"Projected Coordinates in NAD83 Conus Albers (EPSG:5070): X={station_x}, Y={station_y}")

Projected Coordinates in NAD83 Conus Albers (EPSG:5070): X=-1251001.2763440341, Y=1921956.3381891234


In [28]:
foot_xy = {}
#df = pd.read_csv(config['METADATA']['climate_file_path'],skiprows=config['METADATA']['skiprows'])
df = pd.read_csv("../../station_data/US-CdM_HH_202306141100_202410081700.csv")
for col in df.columns:
    if "fetch" in col.lower():
        df = micromet.polar_to_cartesian_dataframe(df, wd_column='WD',dist_column=col)
        df[f'X_{col}'] = np.round(df[f'X_{col}'] + station_x,-1)
        df[f'Y_{col}'] = np.round(df[f'Y_{col}'] + station_y,-1)

        foot_xy[col] = micromet.aggregate_to_daily_centroid(df,'datetime_start',f'X_{col}',f'Y_{col}')


In [6]:
url = "https://services1.arcgis.com/99lidPhWCzftIe9K/arcgis/rest/services/WaterRelatedLandUse/FeatureServer/0/query?where=1%3D1&outFields=*&geometry=-110.75%2C39.438%2C-110.710%2C39.45&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outSR=5070&f=json"

well_fields = gpd.read_file(url)
well_fields = well_fields[well_fields['OBJECTID'].isin([53384,57425])]
well_fields.plot()

In [19]:
# Get the extent of the shapefile
total_bounds = well_fields.total_bounds

# Get minX, minY, maxX, maxY
minX, minY, maxX, maxY = total_bounds

# Create a fishnet
x, y = (minX, minY)
geom_array = []

# Polygon Size
square_size = 20
while y <= maxY:
    while x <= maxX:
        geom = shapely.geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
        geom_array.append(geom)
        x += square_size
    x = minX
    y += square_size

fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:5070')
#fishnet.to_file('fishnet_grid.shp')

In [20]:
fishnet.centroid

0        POINT (-1253535.91 1921252.721)
1        POINT (-1253515.91 1921252.721)
2        POINT (-1253495.91 1921252.721)
3        POINT (-1253475.91 1921252.721)
4        POINT (-1253455.91 1921252.721)
                      ...               
22737    POINT (-1250315.91 1923972.721)
22738    POINT (-1250295.91 1923972.721)
22739    POINT (-1250275.91 1923972.721)
22740    POINT (-1250255.91 1923972.721)
22741    POINT (-1250235.91 1923972.721)
Length: 22742, dtype: geometry

In [24]:
df.columns

Index(['datetime_start', 'TIMESTAMP_START', 'TIMESTAMP_END', 'CO2',
       'CO2_SIGMA', 'H2O', 'H2O_SIGMA', 'FC', 'FC_SSITC_TEST', 'LE',
       'LE_SSITC_TEST', 'ET', 'ET_SSITC_TEST', 'H', 'H_SSITC_TEST', 'G', 'SG',
       'FETCH_MAX', 'FETCH_90', 'FETCH_55', 'FETCH_40', 'WD', 'WS', 'WS_MAX',
       'USTAR', 'ZL', 'TAU', 'TAU_SSITC_TEST', 'MO_LENGTH', 'U', 'U_SIGMA',
       'V', 'V_SIGMA', 'W', 'W_SIGMA', 'PA', 'TA_1_1_1', 'RH_1_1_1',
       'T_DP_1_1_1', 'VPD', 'T_SONIC', 'T_SONIC_SIGMA', 'TS_1_1_1',
       'SWC_1_1_1', 'ALB', 'NETRAD', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'P',
       'file_no', 'datalogger_no', 'TA_1_2_1', 'RH_1_2_1', 'T_DP_1_2_1',
       'TA_1_3_1', 'RH_1_3_1', 'T_DP_1_3_1', 'TA_1_4_1', 'PBLH_F', 'TS_2_1_1',
       'SWC_2_1_1', 'X_FETCH_MAX', 'Y_FETCH_MAX', 'Date', 'X_FETCH_90',
       'Y_FETCH_90', 'X_FETCH_55', 'Y_FETCH_55', 'X_FETCH_40', 'Y_FETCH_40'],
      dtype='object')

In [29]:


# Create a geometry column
df['geometry'] = gpd.points_from_xy(df['X_FETCH_55'], df['Y_FETCH_55'])
df['geometry'].unique()
# Convert the DataFrame to a GeoDataFrame
#gdf = gpd.GeoDataFrame(df, geometry='geometry')

<GeometryArray>
[             <POINT EMPTY>, <POINT (-1251030 1921960)>,
 <POINT (-1250960 1921940)>, <POINT (-1251020 1921940)>,
 <POINT (-1251030 1921920)>, <POINT (-1251020 1921910)>,
 <POINT (-1251040 1921900)>, <POINT (-1251040 1921930)>,
 <POINT (-1251050 1921940)>, <POINT (-1250980 1921980)>,
 ...
 <POINT (-1251050 1922610)>, <POINT (-1251330 1922980)>,
 <POINT (-1251150 1922170)>, <POINT (-1251180 1922190)>,
 <POINT (-1251360 1922010)>, <POINT (-1251570 1922260)>,
 <POINT (-1251680 1922330)>, <POINT (-1252780 1923110)>,
 <POINT (-1251910 1922510)>, <POINT (-1251750 1922450)>]
Length: 4308, dtype: geometry

In [30]:
df

,datetime_start,TIMESTAMP_START,TIMESTAMP_END,CO2,CO2_SIGMA,H2O,H2O_SIGMA,FC,FC_SSITC_TEST,LE,...,X_FETCH_MAX,Y_FETCH_MAX,Date,X_FETCH_90,Y_FETCH_90,X_FETCH_55,Y_FETCH_55,X_FETCH_40,Y_FETCH_40,geometry
0,2023-06-14 11:00:00,202306141100,202306141130,-9999.0000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.000000,...,NaN,NaN,2023-06-14,NaN,NaN,NaN,NaN,NaN,NaN,POINT (NaN NaN)
1,2023-06-14 11:30:00,202306141130,202306141200,-9999.0000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.0,-9999.000000,...,NaN,NaN,2023-06-14,NaN,NaN,NaN,NaN,NaN,NaN,POINT (NaN NaN)
2,2023-06-14 12:00:00,202306141200,202306141230,404.8652,7.160377,11.003180,1.408956,-9999.000000,-9999.0,394.658600,...,-1251020.0,1921960.0,2023-06-14,-1251060.0,1921950.0,-1251030.0,1921960.0,-1251030.0,1921960.0,POINT (-1251030 1921960)
3,2023-06-14 12:30:00,202306141230,202306141300,403.2438,6.578680,10.929970,1.396008,-19.724970,-9999.0,277.552900,...,-1250970.0,1921940.0,2023-06-14,-1250930.0,1921920.0,-1250960.0,1921940.0,-1250970.0,1921940.0,POINT (-1250960 1921940)
4,2023-06-14 13:00:00,202306141300,202306141330,397.2703,7.669390,12.208020,1.357026,-29.465460,-9999.0,290.635000,...,-1251010.0,1921940.0,2023-06-14,-1251040.0,1921920.0,-1251020.0,1921940.0,-1251010.0,1921940.0,POINT (-1251020 1921940)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23143,2024-10-08 14:30:00,202410081430,202410081500,401.5762,0.373631,2.863288,0.053154,-0.284607,-9999.0,5.291050,...,-1250990.0,1921870.0,2024-10-08,-1250970.0,1921700.0,-1250980.0,1921830.0,-1250990.0,1921860.0,POINT (-1250980 1921830)
23144,2024-10-08 15:00:00,202410081500,202410081530,401.5929,0.473523,2.466282,0.107060,-0.755953,2.0,0.461339,...,-1251080.0,1921960.0,2024-10-08,-1251220.0,1921960.0,-1251110.0,1921960.0,-1251090.0,1921960.0,POINT (-1251110 1921960)
23145,2024-10-08 15:30:00,202410081530,202410081600,401.3630,0.335989,2.445982,0.044886,-0.116504,-9999.0,4.989396,...,-1251080.0,1921980.0,2024-10-08,-1251230.0,1922030.0,-1251120.0,1921990.0,-1251090.0,1921980.0,POINT (-1251120 1921990)
23146,2024-10-08 16:00:00,202410081600,202410081630,401.1919,0.380539,2.550954,0.069491,-0.942505,2.0,11.713800,...,-1251070.0,1921940.0,2024-10-08,-1251200.0,1921920.0,-1251100.0,1921940.0,-1251080.0,1921940.0,POINT (-1251100 1921940)
